In [ ]:
import pandas as pd

EVO_pinn = pd.read_csv('data/EVO.csv')

In [9]:
climes = EVO_pinn['Clima'].unique()
print(climes)

['LM' 'LC' 'LS' 'CS' 'CH' 'CSH' 'PLS' 'PLC' 'PLM' 'PPOC' 'PPOR' 'PPC']


In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers


SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Capreta emissions per codecarbon
os.makedirs("emissions", exist_ok=True)

# Funció física de pèrdua.
# Funció pressio de vapor saturació (svp).
def delta_svp_np(T):
    T = np.asarray(T, dtype=float)
    denominator = (T + 237.3)**2
    denominator = np.where(np.abs(denominator) < 1e-9, 1e-9, denominator)
    numerator = 4098 * (0.6108 * np.exp((17.27 * T) / (T + 237.3)))
    delta = numerator / denominator
    return np.nan_to_num(delta, nan=0.0, posinf=0.0, neginf=0.0)

# Funció de Priestley-Taylor
def physics_et0_PT_np(temp, x_orig, alpha=0.82271675438970, gamma=0.1085682687873465, lambda_=2.45):
    Tm = temp
    Rs = x_orig[:, 1]
    delta = delta_svp_np(Tm)
    delta_gamma_sum = delta + gamma
    delta_gamma_sum = np.where(np.abs(delta_gamma_sum) < 1e-9, 1e-9, delta_gamma_sum)
    term1 = delta / delta_gamma_sum
    term1 = np.nan_to_num(term1, nan=0.0, posinf=0.0, neginf=0.0)
    term2 = Rs / lambda_ if lambda_ != 0 else np.zeros_like(Rs)
    term2 = np.nan_to_num(term2, nan=0.0, posinf=0.0, neginf=0.0)
    ET0_PT = alpha * term1 * term2
    return np.maximum(ET0_PT, 0.0)

# Funció de pèrdua per la PINN.
def make_pinn_loss(theta=0.0):
    def pinn_loss(y_true_combined, y_pred):
        y_data_scaled = tf.expand_dims(y_true_combined[:, 0], axis=1)
        y_phys_scaled = tf.expand_dims(y_true_combined[:, 1], axis=1)
        loss_data = tf.reduce_mean(tf.square(y_data_scaled - y_pred))
        loss_phys = tf.reduce_mean(tf.square(y_phys_scaled - y_pred))
        return (1.0 - theta) * loss_data + theta * loss_phys
    return pinn_loss

# Definició model de la xarxa neuronal informada per la física (PINN).
class SimplePINN(Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))
        self.dropout1 = Dropout(0.4)
        self.dense2 = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4))
        self.output_layer = Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dropout1(x, training=training)
        x = self.dense2(x)
        return self.output_layer(x)

# Iteració del model per tipus de clima i valor de theta.
results = []
histories = {}

climes = EVO_pinn['Clima'].unique()
thetas = np.arange(0.2530, 0.2540, 0.0001)

for clima in climes:
    print(f"\n=== Entrenament per clima: {clima} ===")
    df = EVO_pinn[EVO_pinn['Clima'] == clima].copy()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df['Dia'] = pd.to_datetime(df['Dia'])
    df['D'] = df['Dia'].dt.dayofyear
    df['Dia_sin'] = np.sin(2 * np.pi * df['D'] / 365.0)
    df['Dia_cos'] = np.cos(2 * np.pi * df['D'] / 365.0)

    le_clima = LabelEncoder()
    df['Clima_enc'] = le_clima.fit_transform(df['Clima'])

    X_num = df[['Tmax', 'Irradiacio_solar']].values
    y = df['ETo'].values
    temp = df['Tm'].values

    X_num_train, X_num_test, temp_train, temp_test, y_train, y_test = train_test_split(
        X_num, temp, y, test_size=0.2, random_state=SEED
    )

    scaler_X = StandardScaler()
    X_num_train_scaled = scaler_X.fit_transform(X_num_train)
    X_num_test_scaled = scaler_X.transform(X_num_test)

    scaler_y = MinMaxScaler()
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

    ET0_PT_train_orig = physics_et0_PT_np(temp_train, X_num_train)
    ET0_PT_test_orig = physics_et0_PT_np(temp_test, X_num_test)

    ET0_PT_train_scaled = scaler_y.transform(ET0_PT_train_orig.reshape(-1, 1)).flatten()
    ET0_PT_test_scaled = scaler_y.transform(ET0_PT_test_orig.reshape(-1, 1)).flatten()

    y_train_combined = np.hstack([y_train_scaled[:, np.newaxis], ET0_PT_train_scaled[:, np.newaxis]])
    y_test_combined = np.hstack([y_test_scaled[:, np.newaxis], ET0_PT_test_scaled[:, np.newaxis]])

    for theta in thetas:
        print(f"→ Entrenant {clima} amb θ = {theta:.4f}")
        K.clear_session()

        loss_fn = make_pinn_loss(theta=theta)
        model = SimplePINN()
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=loss_fn, metrics=['mae'])

        early_stop = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True, verbose=0)

        tracker = EmissionsTracker(
            project_name=f"{clima}_theta_{theta:.4f}",
            output_dir="emissions"
        )
        tracker.start()
        start_time = time.time()

        history = model.fit(
            x=X_num_train_scaled,
            y=y_train_combined,
            validation_data=(X_num_test_scaled, y_test_combined),
            epochs=300,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )

        elapsed = time.time() - start_time
        emissions = tracker.stop()

        y_pred_scaled = model.predict(X_num_test_scaled).flatten()
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        y_true = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

        rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
        r2_val = r2_score(y_true, y_pred)
        r_val = np.corrcoef(y_true, y_pred)[0, 1]

        results.append({
            "clima": clima,
            "theta": round(theta, 4),
            "rmse": round(rmse_val, 4),
            "r2": round(r2_val, 4),
            "r": round(r_val, 4),
            "emissions_kg": round(emissions, 6),
            "temps_s": round(elapsed, 2)
        })

        histories[(clima, theta)] = history.history
        print(f"✓ θ={theta:.4f} → RMSE: {rmse_val:.4f}, R²: {r2_val:.4f}, CO₂: {emissions:.6f} kg, Temps: {elapsed:.1f}s")

# Resultats.
df_results = pd.DataFrame(results)
df_results.to_csv("/resultas/resultats_EVO_P_PINNS_simple.csv", index=False)



=== Entrenament per clima: LM ===
→ Entrenant LM amb θ = 0.2530


[codecarbon WARNING @ 13:50:29] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 13:50:29] [setup] RAM Tracking...
[codecarbon INFO @ 13:50:29] [setup] CPU Tracking...
[codecarbon WARNING @ 13:50:30] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 13:50:30] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 13:50:30] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 13:50:30] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 13:50:30] [setup] GPU Tracking...
[codecarbon INFO @ 13:50:30] No GPU found.
[codecarbon INFO @ 13:50:30] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
✓ θ=0.2530 → RMSE: 0.2239, R²: 0.9836, CO₂: 0.000064 kg, Temps: 31.7s
→ Entrenant LM amb θ = 0.2531


[codecarbon WARNING @ 13:51:03] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 13:51:03] [setup] RAM Tracking...
[codecarbon INFO @ 13:51:03] [setup] CPU Tracking...
[codecarbon WARNING @ 13:51:04] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 13:51:04] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 13:51:04] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 13:51:04] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 13:51:04] [setup] GPU Tracking...
[codecarbon INFO @ 13:51:04] No GPU found.
[codecarbon INFO @ 13:51:04] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
✓ θ=0.2531 → RMSE: 0.1852, R²: 0.9888, CO₂: 0.000001 kg, Temps: 38.2s
→ Entrenant LM amb θ = 0.2532


...

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
✓ θ=0.2538 → RMSE: 0.1356, R²: 0.9927, CO₂: 0.000069 kg, Temps: 34.3s
→ Entrenant PPC amb θ = 0.2539


[codecarbon WARNING @ 15:43:57] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 15:43:57] [setup] RAM Tracking...
[codecarbon INFO @ 15:43:57] [setup] CPU Tracking...
[codecarbon WARNING @ 15:43:59] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 15:43:59] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 15:43:59] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 15:43:59] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:43:59] [setup] GPU Tracking...
[codecarbon INFO @ 15:43:59] No GPU found.
[codecarbon INFO @ 15:43:59] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
✓ θ=0.2539 → RMSE: 0.1438, R²: 0.9918, CO₂: 0.000084 kg, Temps: 41.4s
→ Entrenant PPC amb θ = 0.2540


[codecarbon WARNING @ 15:44:42] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 15:44:42] [setup] RAM Tracking...
[codecarbon INFO @ 15:44:42] [setup] CPU Tracking...
[codecarbon WARNING @ 15:44:43] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 15:44:43] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 15:44:43] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 15:44:43] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:44:43] [setup] GPU Tracking...
[codecarbon INFO @ 15:44:43] No GPU found.
[codecarbon INFO @ 15:44:43] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
✓ θ=0.2540 → RMSE: 0.1483, R²: 0.9912, CO₂: 0.000080 kg, Temps: 39.6s


In [11]:
print(model.summary())

Model: "simple_pinn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,917 (27.02 KB)

 Trainable params: 2,305 (9.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,612 (18.02 KB)

None
